In [2]:
''' 
To run, first install the pcackages from following code block
Authenticate using your google earth engine credential, replace the term 'ee-kazijahid' with your ee project id. Then authenticate
For downloaded files check at the end, check zour google drive account associated with gee email address
'''

" \nTo run, first install the pcackages from following code block\nAuthenticate using your google earth engine credential, replace the term 'ee-kazijahid' with your ee project id. Then authenticate\n\n"

In [1]:
# !pip install -q geedim
# !pip install -q earthengine-api
# ! pip install -q geemap

In [2]:
# import, authenticate and initialize GEE
import ee
import geedim

# import further necessary packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geemap as emap
from datetime import datetime

ee.Authenticate()
ee.Initialize(project="ee-kazijahid")

Enter verification code: 4/1AeaYSHC14okN7QtxUPUH_J3OLXj4-TBbN7OmbyYeQ0PXUISCUigfPhIni78

Successfully saved authorization token.


In [3]:
# Functions
def mask_s2_clouds(image):
  qa = image.select('QA60')
  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0)))

  return image.updateMask(mask).resample('bicubic').reproject(crs=image.select('B4').projection(), scale=10)



# Define the NDWI function
def addNDWI(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return image.addBands(ndwi)

# Define the NDBI function
def addNDBI(image):
  'Function to calculate ndbi for the provided image. Takes the multiband image as input, and returns a single band image \
    as output. NDBI formula is (B11 -B8)/(B11+B8). collected from https://d-nb.info/1195147821/34'

  ndbi = image.normalizedDifference(['B11', 'B8']).rename('NDBI')  # in sentinel 2  B11 is SWIR and B8 is NIR,

  return image.addBands(ndbi)

def addWaterMask(image):
  NDWI = image.select(['NDWI'])
  return image.addBands(NDWI.gte(0.1).rename('NDWI_Mask'))


  # Function to mask out NDBI
def addBuiltupMask(image):
   NDBI = image.select(['NDBI'])
   return image.addBands(NDBI.gte(0.15).rename('NDBI_Mask'))

def addCombinedMask(image):
  NDWI_Mask = image.select(['NDWI_Mask'])
  NDBI_Mask = image.select(['NDBI_Mask'])
  #  Adds the 2 masks together, and then clamps the output to be binary to keep it suitable for masking
  return image.addBands(NDWI_Mask.add(NDBI_Mask).clamp(0,1).rename('CombinedMask'))


# Function to apply masks and get the final RGB image
def getMaskedFInalImage(image):
    water_mask = image.select('NDWI_Mask')
    builtup_mask = image.select('NDBI_Mask')
    combined_mask = image.select('CombinedMask')

    # Apply masks
    masked_image = image.updateMask(water_mask.Not()).updateMask(builtup_mask.Not()).updateMask(combined_mask.Not())

    return masked_image

# Function to apply masks and get the final binary mask
def getMaskOnly(image):
    water_mask = image.select('WaterMask')
    builtup_mask = image.select('BuiltupMask')
    combined_mask = image.select('CombinedMask')

    # Combine masks using logical OR operation
    binary_mask = combined_mask

    return binary_mask


In [4]:
aoi = ee.FeatureCollection('projects/ee-kazijahid/assets/PhnomPenh_Extended_UTM').geometry().simplify(maxError=50)

# define investigation time period
year_1 = 2019
year_2 = 2023
month_1 =3
month_2 = 6
delta_historical = 3
s2_bands_to_download = [ 'B2', 'B3','B4','NDWI','NDBI','NDWI_Mask','NDBI_Mask','CombinedMask']

In [5]:
s2_y1 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(aoi).filter(ee.Filter.contains(leftField='.geo', rightValue=aoi))\
.filter(ee.Filter.calendarRange(year_1,year_1,'year')).filter(ee.Filter.calendarRange(month_1,month_2,'month'))\
.filter(ee.Filter.rangeContains('CLOUDY_PIXEL_PERCENTAGE', 0, 5)).filter(ee.Filter.lt('DARK_FEATURES_PERCENTAGE', 10)).sort('CLOUDY_PIXEL_PERCENTAGE',True)\
.map(mask_s2_clouds).map(addNDWI).map(addNDBI).map(addWaterMask).map(addBuiltupMask).map(addCombinedMask)


s2_y2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(aoi).filter(ee.Filter.contains(leftField='.geo', rightValue=aoi))\
.filter(ee.Filter.calendarRange(year_2,year_2,'year')).filter(ee.Filter.calendarRange(month_1,month_2,'month'))\
.filter(ee.Filter.rangeContains('CLOUDY_PIXEL_PERCENTAGE', 0, 5)).filter(ee.Filter.lt('DARK_FEATURES_PERCENTAGE', 10)).sort('CLOUDY_PIXEL_PERCENTAGE',True).map(mask_s2_clouds)\
.map(mask_s2_clouds).map(addNDWI).map(addNDBI).map(addWaterMask).map(addBuiltupMask).map(addCombinedMask)

s2_y1_img = s2_y1.first().clip(aoi)
s2_y2_img = s2_y2.first().clip(aoi)


maskedImage_y1 = getMaskedFInalImage(s2_y1_img)
maskedImage_y2 = getMaskedFInalImage(s2_y2_img)

s2_y1_mask = getMaskOnly(s2_y1_img)
s2_y2_mask = getMaskOnly(s2_y2_img)


# create interactive map for visual representations
S2_Map = emap.Map()
S2_Map.add_basemap('SATELLITE')
S2_Map.centerObject(aoi)

S2_Map.addLayer(s2_y1_mask, {'min':0, 'max':1}, 'Y1 Water and BuiltUp Mask')
S2_Map.addLayer(s2_y2_mask, {'min':0, 'max':1}, 'Y2 Water and BuiltUp Mask')
# S2_Map

In [6]:
# emap.download_ee_image(s2_y1_img.select(s2_bands_to_download),  "S2_Y1.tif", scale=10)
# emap.download_ee_image(s2_y2_img.select(s2_bands_to_download),  "S2_Y2.tif", scale=10)


In [7]:
s1_y1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filter(ee.Filter.contains(leftField='.geo', rightValue=aoi))\
    .filter(ee.Filter.eq('instrumentMode','IW')).filter(ee.Filter.listContains('transmitterReceiverPolarisation','VH')).filterMetadata('resolution_meters','equals' ,10)\
    .filter(ee.Filter.calendarRange(year_1,year_1,'year')).filter(ee.Filter.calendarRange(month_1,month_2,'month')).sort('system:time_start')


s1_y2 = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filter(ee.Filter.contains(leftField='.geo', rightValue=aoi))\
    .filter(ee.Filter.eq('instrumentMode','IW')).filter(ee.Filter.listContains('transmitterReceiverPolarisation','VH')).filterMetadata('resolution_meters','equals' ,10)\
    .filter(ee.Filter.calendarRange(year_2,year_2,'year')).filter(ee.Filter.calendarRange(month_1,month_2,'month')).sort('system:time_start')


s1_y1_img = s1_y1.first().clip(aoi)
s1_y2_img = s1_y2.first().clip(aoi)

s1_y1_masked = s1_y1_img.updateMask(s2_y1_mask.Not())
s1_y2_masked = s1_y2_img.updateMask(s2_y2_mask.Not())


s1_y1_stats = s1_y1_masked.select('VV').reduceRegion(reducer=ee.Reducer.minMax(),geometry=aoi,scale=10).getInfo()
s1_y2_stats = s1_y2_masked.select('VV').reduceRegion(reducer=ee.Reducer.minMax(),geometry=aoi,scale=10).getInfo()

vis_params_s1 = {'bands': ['VV','VH', 'VV'], 'min': s1_y1_stats['VV_min'], 'max': s1_y1_stats['VV_max'] ,'gamma':1.0, } #'min': s1_y1_stats['VV_min'], 'max': s1_y1_stats['VV_max'], 'palette':['#ffffff', '#e6cf42', '#eb7308', '#ad366e', '#3d26a6', '#000000']


# create interactive map for visual representations
S1_Map = emap.Map()
S1_Map.add_basemap('SATELLITE')
S1_Map.centerObject(aoi)
S1_Map.addLayer(s1_y1_masked, vis_params_s1, 'S1 Y1 Masked out Image')
S1_Map.addLayer(s1_y2_masked, vis_params_s1, 'S1 Y2 Masked out Image')
# S1_Map


In [8]:
# emap.download_ee_image(s1_y1_img,  "S1_Y1.tif", scale=10)
# emap.download_ee_image(s1_y2_img,  "S1_Y2.tif", scale=10)

# emap.download_ee_image(s1_y1_masked,  "S1_Y1.tif", scale=10)
# emap.download_ee_image(s1_y2_masked,  "S1_Y2.tif", scale=10)

In [10]:
# get the historical lowest value
s1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi)\
    .filter(ee.Filter.eq('instrumentMode','IW'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation','VH'))\
    .filterMetadata('resolution_meters','equals' ,10).select('VV')

# # get the s1 archive start date
# s1_sorted = s1.sort('system:time_start')
# oldest_image = s1_sorted.first()
# oldest_date = oldest_image.date().format("YYYY-MM-dd").getInfo()

s1_y1_date = s1_y1_masked.date().format("YYYY-MM-dd").getInfo()
s1_y2_date = s1_y2_masked.date().format("YYYY-MM-dd").getInfo()




s2_y1_date = s2_y1_img.date().format("YYYY-MM-dd").getInfo()
s2_y2_date = s2_y2_img.date().format("YYYY-MM-dd").getInfo()


# print(f'The S1 Image acquisition of Oldest image is {oldest_date}')
print(f'The S1 Image acquisition of {year_1} image is {s1_y1_date}')
print(f'The S1 Image acquisition of {year_2} image is {s1_y2_date}')

print(f'The S2 Image acquisition of {year_1} image is {s2_y1_date}')
print(f'The S2 Image acquisition of {year_2} image is {s2_y2_date}')


s1_y1_ref_yr=  s1_y1_img.date().get('year').getInfo()-delta_historical
s1_y2_ref_yr = s1_y2_img.date().get('year').getInfo()-delta_historical

# Getting images from last 3 years, to get historcial data
s1_y1_ref_img = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filter(ee.Filter.contains(leftField='.geo', rightValue=aoi))\
    .filter(ee.Filter.eq('instrumentMode','IW')).filter(ee.Filter.listContains('transmitterReceiverPolarisation','VH')).filterMetadata('resolution_meters','equals' ,10)\
    .filter(ee.Filter.calendarRange(s1_y1_ref_yr, year_1,'year')).filter(ee.Filter.calendarRange(1,12,'month')).sort('system:time_start')

s1_y2_ref_img = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filter(ee.Filter.contains(leftField='.geo', rightValue=aoi))\
    .filter(ee.Filter.eq('instrumentMode','IW')).filter(ee.Filter.listContains('transmitterReceiverPolarisation','VH')).filterMetadata('resolution_meters','equals' ,10)\
    .filter(ee.Filter.calendarRange(s1_y2_ref_yr, year_2,'year')).filter(ee.Filter.calendarRange(1,12,'month')).sort('system:time_start')



The S1 Image acquisition of 2019 image is 2019-03-01
The S1 Image acquisition of 2023 image is 2023-03-06
The S2 Image acquisition of 2019 image is 2019-03-12
The S2 Image acquisition of 2023 image is 2023-03-06


In [13]:
print(f'Total Images in 2019: {s1_y1_ref_img.size().getInfo()}')
print(f'Total Images in 2023: {s1_y2_ref_img.size().getInfo()}')

print(f'Total Images in 2019: {s1_y1_img.id().getInfo()}')
print(f'Total Images in 2023: {s1_y2_img.id().getInfo()}')

Total Images in 2019: 295
Total Images in 2023: 316
Total Images in 2019: S1B_IW_GRDH_1SDV_20190301T225231_20190301T225307_015167_01C5DB_74AC
Total Images in 2023: S1A_IW_GRDH_1SDV_20230306T111209_20230306T111234_047523_05B4DE_B426


In [14]:
dry_ref_y1 = s1_y1_ref_img.min().updateMask(s2_y1_mask.Not()) # historical_min_composit
wet_ref_y1 = s1_y1_ref_img.max().updateMask(s2_y1_mask.Not()) # historical_max_composit
sensitivity_y1 = wet_ref_y1.subtract(dry_ref_y1).clip(aoi).select('VV')


dry_ref_y2 = s1_y2_ref_img.min().updateMask(s2_y2_mask.Not()) # historical_min_composit
wet_ref_y2 = s1_y2_ref_img.max().updateMask(s2_y2_mask.Not()) # historical_max_composit
sensitivity_y2 = wet_ref_y2.subtract(dry_ref_y2).clip(aoi).select('VV')

vis_params_s1 = {'bands': ['VV'],'min': 4,'max': 100, 'palette': ['#30123b', '#28bceb', '#a4fc3c', '#fb7e21', '#7a0403']}  #sentivity: VV_max:95.64165945106045, VV_min:5.483494778666355

S1_Map.addLayer(sensitivity_y1, vis_params_s1, 'S1 Y1 Sensitivity')
S1_Map.addLayer(sensitivity_y2, vis_params_s1, 'S1 Y2 Sensitivity')

# S1_Map

In [15]:
vis_params_ssm = {'min': 0, 'max': 1, 'palette':['#ff0000', '#ffb462', '#80ffb4', '#00b4ec', '#8000ff']}
ssm_y1 = (s1_y1_masked.select('VV').subtract(dry_ref_y1.select('VV'))).divide(sensitivity_y1.select('VV')).select('VV')
ssm_y2 = (s1_y2_masked.select('VV').subtract(dry_ref_y2.select('VV'))).divide(sensitivity_y2.select('VV')).select('VV')

ssm_change = ssm_y2.select('VV').subtract(ssm_y1.select('VV'))


S1_Map.addLayer(ssm_y1.clip(aoi), vis_params_ssm, 'S1 Y1 SSM')
S1_Map.addLayer(ssm_y2.clip(aoi), vis_params_ssm, 'S1 Y2 SSM')
S1_Map.addLayer(ssm_y2.clip(aoi), vis_params_ssm, 'SSM Change')

S1_Map

Map(center=[11.579404697350043, 104.88152039722425], controls=(WidgetControl(options=['position', 'transparent…

In [16]:
projection = s1_y1_masked.select('VV').projection().getInfo()['transform']

In [17]:
# The large size images are difficult to download with geemap, so using task
task1 = ee.batch.Export.image.toDrive(
    image=ssm_y1.float(),
    description='SSM_Y1_new',
    folder='RadarChangeDetectionOutPut',
    region=aoi,
    crsTransform=projection,
    crs='EPSG:32648',
    maxPixels=1e13
)

task2 = ee.batch.Export.image.toDrive(
    image=ssm_y2.float(),
    description='SSM_Y2_new',
    folder='RadarChangeDetectionOutPut',
    region=aoi,
    crsTransform=projection,
    crs='EPSG:32648',
    maxPixels=1e13
)

task3 = ee.batch.Export.image.toDrive(
    image=ssm_change.float(),
    description='ssm_change',
    folder='RadarChangeDetectionOutPut',
    region=aoi,
    crsTransform=projection,
    crs='EPSG:32648',
    maxPixels=1e13
)

task4 = ee.batch.Export.image.toDrive(
    image=sensitivity_y1.float(),
    description='sensitivity_y1',
    folder='RadarChangeDetectionOutPut',
    region=aoi,
    crsTransform=projection,
    crs='EPSG:32648',
    maxPixels=1e13
)
task5 = ee.batch.Export.image.toDrive(
    image=sensitivity_y2.float(),
    description='sensitivity_y2',
    folder='RadarChangeDetectionOutPut',
    region=aoi,
    crsTransform=projection,
    crs='EPSG:32648',
    maxPixels=1e13
)

# task1.start()
# task2.start()
# task3.start()
# task4.start()
# task5.start()
